In [18]:
import os
import pandas as pd
from dotenv import load_dotenv
from FiinQuantX import FiinSession
from constant import *

load_dotenv()
username = os.getenv('username')
password = os.getenv('password')



client = FiinSession(
    username=username,
    password=password,
).login()

tickers = VNINDEX # list of tickers: ['UPCOMINDEX',VNINDEX,HNXINDEX]





In [ ]:
data = client.Fetch_Trading_Data(
    realtime = False, 
    tickers = tickers,
    fields = ['open'], 
    from_date='2023-10-01 9:00',
    to_date='2023-11-01 15:00',
    by = '1d',
).get_data()

df = pd.DataFrame(data)
df.to_csv('vnindex_.csv', index=False)

In [55]:
fi = client.FiinIndicator()
data['sma_20'] = fi.sma(data['close'], window = 20)
data['sma_50'] = fi.sma(data['close'], window = 50)
data['sma_200'] = fi.sma(data['close'], window = 200)
print(data[['close', 'sma_20', 'sma_50', 'sma_200']])

            close   sma_20   sma_50       sma_200
0    14890.248248      NaN      NaN           NaN
1    15077.841927      NaN      NaN           NaN
2    15007.494298      NaN      NaN           NaN
3    15007.494298      NaN      NaN           NaN
4    15030.943507      NaN      NaN           NaN
..            ...      ...      ...           ...
995  26200.000000  25212.5  23345.0  21625.276500
996  27350.000000  25442.5  23468.0  21658.633375
997  27650.000000  25675.0  23595.0  21692.028125
998  27550.000000  25902.5  23720.0  21724.922875
999  27800.000000  26142.5  23847.0  21759.903125

[1000 rows x 4 columns]


In [58]:
class backtest_strategy:
    def __init__(self, data):
        self.data = data
        self.position = 0  # 1 for holding a position, 0 for no position
        self.cash = 100000  # starting cash
        self.shares = 0  # number of shares held
        self.trades = []  # record of trades

    def run(self):
        for i in range(1, len(self.data)):
            if self.data['sma_20'].iloc[i] > self.data['sma_50'].iloc[i] and self.position == 0:
                # Buy signal
                self.shares = self.cash // self.data['close'].iloc[i]
                self.cash -= self.shares * self.data['close'].iloc[i]
                self.position = 1
                self.trades.append((self.data.index[i], 'BUY', self.shares, self.data['close'].iloc[i]))
                print(f"[BUY] Date: {self.data['timestamp'].iloc[i]}, Shares: {self.shares}, Price: {self.data['close'].iloc[i]}, Cash left: {self.cash}")
            elif self.data['sma_20'].iloc[i] < self.data['sma_50'].iloc[i] and self.position == 1:
                # Sell signal
                self.cash += self.shares * self.data['close'].iloc[i]
                self.position = 0
                self.trades.append((self.data.index[i], 'SELL', self.shares, self.data['close'].iloc[i]))
                print(f"[SELL] Date: {self.data['timestamp'].iloc[i]}, Shares: {self.shares}, Price: {self.data['close'].iloc[i]}, Cash after sale: {self.cash}")
                self.shares = 0

        # Final portfolio value
        final_value = self.cash + (self.shares * self.data['close'].iloc[-1])
        return final_value, self.trades

bt = backtest_strategy(data)
final_value, trades = bt.run()
print(f"Final Portfolio Value: {final_value}")

[BUY] Date: 2021-11-08 00:00, Shares: 6.0, Price: 15664.07217312, Cash left: 6015.566961280012
[SELL] Date: 2021-12-27 00:00, Shares: 6.0, Price: 15546.82612392, Cash after sale: 99296.52370480001
[BUY] Date: 2022-01-04 00:00, Shares: 6.0, Price: 16133.05636992, Cash left: 2498.185485280017
[SELL] Date: 2022-01-06 00:00, Shares: 6.0, Price: 15828.216642, Cash after sale: 97467.48533728001
[BUY] Date: 2022-01-24 00:00, Shares: 6.0, Price: 16109.60716008, Cash left: 809.8423768000066
[SELL] Date: 2022-03-11 00:00, Shares: 6.0, Price: 15335.78323536, Cash after sale: 92824.54178896
[BUY] Date: 2022-07-21 00:00, Shares: 6.0, Price: 14450.5755639, Cash left: 6121.088405560004
[SELL] Date: 2022-07-25 00:00, Shares: 6.0, Price: 14098.8374163, Cash after sale: 90714.11290336002
[BUY] Date: 2022-08-08 00:00, Shares: 6.0, Price: 14509.1985885, Cash left: 3658.921372360026
[SELL] Date: 2022-09-16 00:00, Shares: 6.0, Price: 13571.2301949, Cash after sale: 85086.30254176003
[BUY] Date: 2022-12-02 0